In [27]:
import bz2
import lzma
import json
import pickle 
import os
import multiprocessing
import gzip

In [10]:
def worker(file_path):
    print(file_path[-8:-3])
    
    subreddits = {}
    submissioners = {}
    
    if '.bz2' in file_path:
        _open = bz2.open
    else:
        _open = lzma.open
    
    with _open(file_path,'rt') as submissions_data:
        for i,submission in enumerate(submissions_data):
            submission = json.loads(submission)
            if 'subreddit' in submission:
                if submission['subreddit'] not in subreddits:
                    subreddits[submission['subreddit']] = {
                        'submissions': {},
                        'submissioners': set()
                    }
                if submission['author'] not in subreddits[submission['subreddit']]['submissions']:
                    subreddits[submission['subreddit']]['submissions'][submission['author']] = {
                        'timestamps': [],
                        'votes': [],
                        'comments': []
                    }
                    
                subreddits[submission['subreddit']]['submissions'][submission['author']]['timestamps'].append(submission['created_utc'])
                subreddits[submission['subreddit']]['submissions'][submission['author']]['votes'].append(submission['score'])
                subreddits[submission['subreddit']]['submissions'][submission['author']]['comments'].append(submission['num_comments'])
                subreddits[submission['subreddit']]['submissioners'].add(submission['author'])
                
                if submission['author'] not in submissioners:
                    submissioners[submission['author']] = {
                        'timestamps': [],
                        'votes': [],
                        'comments': []
                    }
                    
                submissioners[submission['author']]['timestamps'].append(submission['created_utc'])
                submissioners[submission['author']]['votes'].append(submission['score'])
                submissioners[submission['author']]['comments'].append(submission['num_comments'])
                
            if i % 1000000 == 0:
                print(file_path,i)

        #save the subreddit dict in a file
        pickle_out = open("Results/submissions_" + file_path[-8:-3] + ".pickle","wb")
        pickle.dump(subreddits, pickle_out)
        pickle_out.close()
        
        #save the subreddit dict in a file
        pickle_out = open("Results/submissioners_" + file_path[-8:-3] + ".pickle","wb")
        pickle.dump(submissioners, pickle_out)
        pickle_out.close()

# path storing the data regarding the submissions
folder = '/home/data/reddit/submissions/'
files = os.listdir(folder)
required_files = []


for key,file in enumerate(files):
    if "v2" not in file  and "2017-07" not in file and "2017-11" not in file and file[3:7] >= "2017" and ('xz' in file):
        required_files.append(folder + file)
sorted(required_files)

['/home/data/reddit/submissions/RS_2017-12.xz',
 '/home/data/reddit/submissions/RS_2018-01.xz',
 '/home/data/reddit/submissions/RS_2018-02.xz',
 '/home/data/reddit/submissions/RS_2018-03.xz',
 '/home/data/reddit/submissions/RS_2018-04.xz',
 '/home/data/reddit/submissions/RS_2018-05.xz',
 '/home/data/reddit/submissions/RS_2018-06.xz',
 '/home/data/reddit/submissions/RS_2018-07.xz',
 '/home/data/reddit/submissions/RS_2018-08.xz']

In [11]:
# run the calculation
p = multiprocessing.Pool(len(required_files))
print(p.map(worker, required_files))
p.close()

18-05
18-06
17-12
18-04
18-02
18-07
18-08
18-03
18-01
/home/data/reddit/submissions/RS_2017-12.xz 0
/home/data/reddit/submissions/RS_2018-05.xz 0
/home/data/reddit/submissions/RS_2018-02.xz 0
/home/data/reddit/submissions/RS_2018-08.xz 0
/home/data/reddit/submissions/RS_2018-06.xz 0
/home/data/reddit/submissions/RS_2018-07.xz 0
/home/data/reddit/submissions/RS_2018-04.xz 0
/home/data/reddit/submissions/RS_2018-01.xz 0
/home/data/reddit/submissions/RS_2018-03.xz 0
/home/data/reddit/submissions/RS_2018-01.xz 1000000
/home/data/reddit/submissions/RS_2018-02.xz 1000000
/home/data/reddit/submissions/RS_2017-12.xz 1000000
/home/data/reddit/submissions/RS_2018-03.xz 1000000
/home/data/reddit/submissions/RS_2018-04.xz 1000000
/home/data/reddit/submissions/RS_2018-07.xz 1000000
/home/data/reddit/submissions/RS_2018-08.xz 1000000
/home/data/reddit/submissions/RS_2018-06.xz 1000000
/home/data/reddit/submissions/RS_2018-05.xz 1000000
/home/data/reddit/submissions/RS_2018-01.xz 2000000
/home/data/r

In [25]:
required_files = []
for file in os.listdir('Results'):
    if 'submissions' in file:
        required_files.append(file[-15:-10])
sorted(required_files)

['16-01',
 '16-02',
 '16-03',
 '16-04',
 '16-05',
 '16-06',
 '16-07',
 '16-08',
 '16-09',
 '16-10',
 '16-11',
 '16-12',
 '17-01',
 '17-02',
 '17-03',
 '17-04',
 '17-05',
 '17-06',
 '17-07',
 '17-08',
 '17-09',
 '17-10',
 '17-11',
 '17-12',
 '18-01',
 '18-02',
 '18-03',
 '18-04',
 '18-05',
 '18-06',
 '18-07',
 '18-08']

In [32]:
subreddits = {}
submissioners = {}

for file in required_files:
    submissioners_temp = pickle.load(gzip.open("Results/submissioners_" + file + ".pickle.gz","rb"))
    subreddits_temp = pickle.load(gzip.open("Results/submissions_" + file + ".pickle.gz","rb"))
    
    for submissioner in submissioners_temp:
        
        if submissioner not in submissioners:
            submissioners[submissioner] = {
                'timestamps': [],
                'votes': [],
                'comments': []
                        
            }
            
        submissioners[submissioner]['timestamps'].extend(
            submissioners_temp[submissioner]['timestamps']
        )
        submissioners[submissioner]['votes'].extend(
            submissioners_temp[submissioner]['votes']
        )
        submissioners[submissioner]['comments'].extend(
            submissioners_temp[submissioner]['comments']
        )
        
    for subreddit in subreddits_temp:
        
        if subreddit not in subreddits:
            subreddits[subreddit] = {
                'submissions': {},
                'submissioners': set()
            }
        
        subreddits[subreddit]['submissioners'].update(
            subreddits_temp[subreddit]['submissioners']
        )
        
        for submissioner in subreddits_temp[subreddit]['submissions']:
            
            if submissioner not in subreddits[subreddit]['submissions']:
                subreddits[subreddit]['submissions'][submissioner] = {
                    'timestamps': [],
                    'votes': [],
                    'comments': []
                }
                
            subreddits[subreddit]['submissions'][submissioner]['timestamps'].extend(
                subreddits_temp[subreddit]['submissions'][submissioner]['timestamps']
            )
            
            subreddits[subreddit]['submissions'][submissioner]['votes'].extend(
                subreddits_temp[subreddit]['submissions'][submissioner]['votes']
            )
            
            subreddits[subreddit]['submissions'][submissioner]['comments'].extend(
                subreddits_temp[subreddit]['submissions'][submissioner]['comments']
            )
    print(len(submissioners))
    print(len(subreddits['politics']['submissioners']))

                        
pickle_out = open("Results/submissioners.pickle","wb")
pickle.dump(submissioners, pickle_out)
pickle_out.close()

pickle_out = open("Results/submissions.pickle","wb")
pickle.dump(subreddits, pickle_out)
pickle_out.close()

2365246
6416
3866315
16154
5037508
19591
5976153
25557
6630617
30256
7593964
34842
8219978
39411
8695993
43422
9477313
47645
9914433
53767
10982563
59552
11981217
62801
12411867
66204
12968438
69731
13471276
74470
14090585
76993
14554455
81474
15001483
84238
15560849
87629
15888766
90825
16184949
96778
16580826
100722
16897121
103169
17329084
105783
17662637
108245
18028376
111141
18346810
115183
18842216
117862
19265420
119889
19587750
121833
19869314
124248
20226667
127227
